In [3]:
pip install numexpr


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 91 kB 1.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [5]:
import os
os.environ["OPENAI_API_KEY"] = "sk-***"
import openai
openai.api_key = "sk-***"

In [7]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage

try:
    storage_context = StorageContext.from_defaults(persist_dir='/Users/luizbueno/Downloads/LamaIndex RAG/data')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('data').load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir='/Users/luizbueno/Downloads/LamaIndex RAG/data')
    print('persisting to disk')

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
loading from disk


In [8]:
openai.log = "debug"

In [23]:
from llama_index.prompts import PromptTemplate

text_qa_template_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    """Context:
    Timestamp: {timestamp}
    Time: {time}
    Day period: {day_period}
    Day of Week: {day_of_week}
    Month Day: {month_day}
    Month: {month}
    Dew Point Process: {dew_point_process}
    Dew Point Contactor: {dew_point_contactor}
    Pressure Process Contactor: {pressure_process_contactor}
    Pressure Natural Gas Moisture Process: {pressure_natural_gas_moisture_process}
    Natural Gas Moisture Contactor: {natural_gas_moisture_contactor}
    Temperature Process Contactor: {temperature_process_contactor}
    Glycol Moisture Process: {glycol_moisture_process}
    Glycol Moisture Contactor: {glycol_moisture_contactor}
    Water Inlet Temperature Process: {water_inlet_temperature_process}
    Water Inlet Temperature Glycol: {water_inlet_temperature_glycol}
    Inlet Temperature Process Glycol: {inlet_temperature_process_glycol}
    Out Glycol Temperature Process: {out_glycol_temperature_process}
    Out Glycol Temperature: {out_glycol_temperature}
    Temperature Process: {temperature_process}
    Out Water Temperature Process: {out_water_temperature_process}
    Out Water Temperature: {out_water_temperature}
    Stripping Gas Process: {stripping_gas_process}
    Stripping Gas Pressure Process: {stripping_gas_pressure_process}
    Pressure Dry Glycol Process: {pressure_dry_glycol_process}
    Dry Glycol Process: {dry_glycol_process}
    Glycol Flow Process: {glycol_flow_process}

    Questions:
    1. What was the Dew Point Process at {timestamp}?
    2. What was the Day of Week on {timestamp}?
    3. What is the Pressure Natural Gas Moisture Process at {timestamp}?
    4. ...

    """
    "answer the question: {query_str}\n"
    "If the context isn't helpful, you can also answer the question on your own.\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [24]:
text_qa_template_str = (
    """
    "timestamp": "11-01-2023 00:02:00",
    "time": "Night",
    "day_period": "Wednesday",
    "day_of_week": "11 January",
    "month_day": "-5.621632",
    "month": "Don't Criticize",
    "dew_point_process": "-5.621632",
    "dew_point_contactor": "Don't Criticize",
    "pressure_process_contactor": "161.305446",
    "pressure_natural_gas_moisture_process": "Low",
    "natural_gas_moisture_contactor": "2.71",
    "temperature_process_contactor": "Normal",
    "glycol_moisture_process": "42.6",
    "glycol_moisture_contactor": "Low",
    "water_inlet_temperature_process": "0.67",
    "water_inlet_temperature_glycol": "Efficient",
    "inlet_temperature_process_glycol": "34.56",
    "out_glycol_temperature_process": "Normal",
    "out_glycol_temperature": "52.67",
    "temperature_process": "Low",
    "out_water_temperature_process": "40.75",
    "out_water_temperature": "Good",
    "stripping_gas_process": "183.626183",
    "stripping_gas_pressure_process": "Keep",
    "pressure_dry_glycol_process": "58.355965",
    "dry_glycol_process": "Bad",
    "glycol_flow_process": "107.583807", """
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Answer the question: {query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [25]:
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("How was Dew Point on 01-09-2023?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [26]:
print(response)

The Dew Point on 01-09-2023 was -3.075649.


In [27]:
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("Give me the complete report of the incidents on 01-09-2023?")
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
On 01-09-2023, there was one incident reported. Here are the details of the incident:

- Timestamp: 01-09-2023 00:00:00
- Time: Night
- Day period: Friday
- Day of Week: 1 September
- Month Day: -3.075649
- Month: Don't Criticize
- Dew Point Process: -3.075649
- Dew Point Contactor: Don't Criticize
- Pressure Process Contactor: 172.956649
- Pressure Natural Gas Moisture Process: Low
- Natural Gas Moisture Contactor: 1.38
- Temperature Process Contactor: Efficient
- Glycol Moisture Process: 34.78
- Glycol Moisture Contactor: Normal
- Water Inlet Temperature Process: 52.05
- Water Inlet Temperature Glycol: Low
- Inlet Temperature Process Glycol: 47.64
- Out Glycol

In [29]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='304117a8-8296-4fd7-a608-abf1294ed909', embedding=None, metadata={'page_label': '1', 'file_name': 'df_100_PDF.pdf', 'file_path': 'data/df_100_PDF.pdf', 'file_type': 'application/pdf', 'file_size': 88384, 'creation_date': '2024-01-17', 'last_modified_date': '2024-01-09', 'last_accessed_date': '2024-01-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bdb949d7-be97-4364-b7df-7cdf2ec6c91a', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'df_100_PDF.pdf', 'file_path': 'data/df_100_PDF.pdf', 'file_type': 'application/pdf', 'file_size': 88384, 'creation_date': '2024-01-17', 'last_modified_date': '2024-01-09', 'last_accessed_date': '2024-01-17'}

In [30]:
from llama_index.response.pprint_utils import pprint_response
pprint_response(response, show_source=True)

Final Response: On 01-09-2023, there was one incident reported. Here
are the details of the incident:  - Timestamp: 01-09-2023 00:00:00 -
Time: Night - Day period: Friday - Day of Week: 1 September - Month
Day: -3.075649 - Month: Don't Criticize - Dew Point Process: -3.075649
- Dew Point Contactor: Don't Criticize - Pressure Process Contactor:
172.956649 - Pressure Natural Gas Moisture Process: Low - Natural Gas
Moisture Contactor: 1.38 - Temperature Process Contactor: Efficient -
Glycol Moisture Process: 34.78 - Glycol Moisture Contactor: Normal -
Water Inlet Temperature Process: 52.05 - Water Inlet Temperature
Glycol: Low - Inlet Temperature Process Glycol: 47.64 - Out Glycol
Temperature Process: Good - Out Glycol Temperature: 175.224011 -
Temperature Process: Not Keep - Out Water Temperature Process:
43.727277 - Out Water Temperature: Bad - Stripping Gas Process:
96.216138 - Stripping Gas Pressure Process: Normal - Pressure Dry
Glycol Process: 29.568994 - Dry Glycol Process: Normal 